<a href="https://colab.research.google.com/github/laroccathebrux/ciencia_consumo/blob/main/Questions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pyspark

     |████████████████████████████████| 212.4MB 70kB/s 
     |████████████████████████████████| 204kB 19.8MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=27e022d0633f228fa323747d934c12d58ff487873ebf1f5af9165c4a45f627ec
  Stored in directory: /root/.cache/pip/wheels/40/1b/2c/30f43be2627857ab80062bef1527c0128f7b4070b6b2d02139
Successfully built pyspark


In [6]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
import psycopg2

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [4]:
spark = SparkSession \
    .builder \
    .appName("Desafio Engenheiro de Dados") \
    .getOrCreate()

In [7]:
conn = psycopg2.connect("host=batyr.db.elephantsql.com dbname=hzlsgeqq user=hzlsgeqq password=")
cur = conn.cursor()

In [8]:
cur.execute(
    "SELECT \
      p.nome AS pais, \
      count(r.id) AS qtd_respondentes \
    FROM \
      respondente AS r \
    INNER JOIN \
      pais AS p ON p.id = r.pais_id \
    GROUP BY pais \
    ORDER BY qtd_respondentes DESC"
    )
rst = cur.fetchall()
df_rst_r1 = spark.createDataFrame(rst, ["pais", "qtd_respondentes"])
df_rst_r1.show()

+--------------------+----------------+
|                pais|qtd_respondentes|
+--------------------+----------------+
|       United States|            2350|
|               India|            1124|
|      United Kingdom|             749|
|             Germany|             655|
|              Canada|             360|
|              France|             278|
|  Russian Federation|             270|
|              Brazil|             255|
|              Poland|             233|
|               Spain|             203|
|           Australia|             194|
|         Netherlands|             193|
|               Italy|             166|
|              Sweden|             129|
|             Ukraine|             109|
|         Switzerland|             107|
|              Israel|             101|
|              Turkey|              98|
|               China|              92|
|Iran, Islamic Rep...|              89|
+--------------------+----------------+
only showing top 20 rows



In [9]:
cur.execute(
    "SELECT \
      p.nome AS pais, \
      count(r.id) AS qtd_respondentes \
    FROM \
      respondente AS r \
    INNER JOIN \
      pais AS p ON p.id = r.pais_id \
    INNER JOIN \
      sistema_operacional AS so ON so.id = r.sistema_operacional_id \
    WHERE \
      so.nome = 'Windows' \
    AND p.nome = 'United States'\
    GROUP BY pais \
    ORDER BY qtd_respondentes DESC"
    )
rst = cur.fetchall()
df_rst_r2 = spark.createDataFrame(rst, ["pais", "qtd_respondentes"])
df_rst_r2.show()

+-------------+----------------+
|         pais|qtd_respondentes|
+-------------+----------------+
|United States|             961|
+-------------+----------------+



In [10]:
sql = (
    "SELECT \
      p.nome AS pais, \
      AVG(r.salario) AS media_salario \
    FROM \
      respondente AS r \
    INNER JOIN \
      pais AS p ON p.id = r.pais_id \
    INNER JOIN \
      sistema_operacional AS so ON so.id = r.sistema_operacional_id \
    WHERE \
      so.nome = 'Linux-based' \
    AND p.nome = 'Israel' \
    GROUP BY \
      pais"
)
cur.execute(sql)
rst = cur.fetchall()
df_rst_r3 = spark.createDataFrame(rst, ["pais", "media_salario"])
df_rst_r3 = df_rst_r3.withColumn("media_salario", f.round(f.col("media_salario"),2))
df_rst_r3.show()

+------+-------------+
|  pais|media_salario|
+------+-------------+
|Israel|     28335.34|
+------+-------------+



In [11]:
sql = (
    "SELECT \
      e.tamanho, \
      AVG(r.salario) AS media_salario, \
      STDDEV(r.salario) AS desvio_padrao_salario \
    FROM \
      respondente AS r \
    INNER JOIN \
      empresa AS e ON e.id = r.empresa_id \
    INNER JOIN \
      resp_usa_ferramenta as ruf ON ruf.respondente_id = r.id \
    INNER JOIN \
      ferramenta_comunic as fc ON fc.id = ruf.ferramenta_comunic_id \
    WHERE \
      fc.nome = 'Slack' \
    GROUP BY \
      e.tamanho \
    ORDER BY \
      e.tamanho DESC \
    "
)
#print(sql)
cur.execute(sql)

rst = cur.fetchall()
df_rst_r3 = spark.createDataFrame(rst, ["tamanho", "media_salario", "desvio_padrao_salario"])
df_rst_r3 = df_rst_r3.withColumn("media_salario", f.round(f.col("media_salario"),2))
df_rst_r3 = df_rst_r3.withColumn("desvio_padrao_salario", f.round(f.col("desvio_padrao_salario"),2))
df_rst_r3.show()

+--------------------+-------------+---------------------+
|             tamanho|media_salario|desvio_padrao_salario|
+--------------------+-------------+---------------------+
|Fewer than 10 emp...|     35948.71|             79944.27|
|500 to 999 employees|     41505.91|             69678.48|
|5,000 to 9,999 em...|     44278.03|              90114.5|
|  20 to 99 employees|     44218.36|             92959.44|
|  10 to 19 employees|     35969.82|             80785.93|
|100 to 499 employees|     45347.78|             85130.77|
|1,000 to 4,999 em...|     45302.27|             75904.22|
|10,000 or more em...|     56273.84|            113219.31|
+--------------------+-------------+---------------------+

